<a href="https://colab.research.google.com/github/adnajihad/sentiment_analysis-moviereview/blob/main/kerjain_TA_anj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/adnajihad/sentiment_analysis-moviereview

Cloning into 'sentiment_analysis-moviereview'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.


In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [3]:
data = pd.read_csv('/content/sentiment_analysis-moviereview/valid.csv')
data.head(3)

,review,label
0,"Encouraged by the positive comments about this film on here I was looking forward to watching this film. Bad mistake. I've seen 950+ films and this is truly one of the worst of them - it's awful in almost every way: editing, pacing, storyline, 'acting,' soundtrack (the film's only song - a lame country tune - is played no less than four times). The film looks cheap and nasty and is boring in the extreme. Rarely have I been so happy to see the end credits of a film. <br /><br />The only thing that prevents me giving this a 1-score is Harvey Keitel - while this is far from his best performance he at least seems to be making a bit of an effort. One for Keitel obsessives only.",0
1,"If you like original gut wrenching laughter you will like this movie. If you are young or old then you will love this movie, hell even my mom liked it.<br /><br />Great Camp!!!",1
2,"Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines.<br /><br />At first it was very odd and pretty funny but as the movie progressed I didn't find the jokes or oddness funny anymore.<br /><br />Its a low budget film (thats never a problem in itself), there were some pretty interesting characters, but eventually I just lost interest.<br /><br />I imagine this film would appeal to a stoner who is currently partaking.<br /><br />For something similar but better try ""Brother from another planet""",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  7000 non-null   object
 1   label   7000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 109.5+ KB


# PreProcessing

## Cleaning

In [5]:
import string

#remove <br/>
removed_br = []

for review in data['review']:
  removed_br.append(review.replace('<br /><br />', ' '))

data['review'] = removed_br

#removing special characters
def cleaning(s):
    clear_string = ""
    for symbol in s:
        if symbol not in string.punctuation:
            clear_string += symbol
    return clear_string

#removing certain characters
string_punctuation =  "\n"

def remove(s):
    no_punct = ""
    for letter in s:
        if letter not in string_punctuation:
            no_punct += letter
    return no_punct

#make sure everything is clean for the last time
def preprocess(dat):
    #Ubah - menjadi space
    regex = r'\-'
    dat = dat.replace(regex, ' ')
    regex = r'[^a-zA-Z\s]' 

    # Ubah seluruh huruf dalam data artikel ke dalam lower case(casefolding)
    dat = dat.lower()

    # Hapus seluruh simbol yang ada dan ganti dengan string kosong ('')
    dat = dat.replace(regex, '')

    # Hapus enter, ganti dengan space
    regex = r'\n'
    dat = dat.replace(regex, ' ')

    return dat

data['review_clean'] = data['review'].apply(cleaning)
data['review_clean'] = data['review_clean'].apply(remove)
data['review_clean'] = data['review_clean'].apply(preprocess)

#removing numbers
data['review_clean'] = data['review_clean'].str.replace(r'\d+','')

data = data[['review','review_clean','label']]

data.head(3)

,review,review_clean,label
0,"Encouraged by the positive comments about this film on here I was looking forward to watching this film. Bad mistake. I've seen 950+ films and this is truly one of the worst of them - it's awful in almost every way: editing, pacing, storyline, 'acting,' soundtrack (the film's only song - a lame country tune - is played no less than four times). The film looks cheap and nasty and is boring in the extreme. Rarely have I been so happy to see the end credits of a film. The only thing that prevents me giving this a 1-score is Harvey Keitel - while this is far from his best performance he at least seems to be making a bit of an effort. One for Keitel obsessives only.",encouraged by the positive comments about this film on here i was looking forward to watching this film bad mistake ive seen films and this is truly one of the worst of them its awful in almost every way editing pacing storyline acting soundtrack the films only song a lame country tune is played no less than four times the film looks cheap and nasty and is boring in the extreme rarely have i been so happy to see the end credits of a film the only thing that prevents me giving this a score is harvey keitel while this is far from his best performance he at least seems to be making a bit of an effort one for keitel obsessives only,0
1,"If you like original gut wrenching laughter you will like this movie. If you are young or old then you will love this movie, hell even my mom liked it. Great Camp!!!",if you like original gut wrenching laughter you will like this movie if you are young or old then you will love this movie hell even my mom liked it great camp,1
2,"Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines. At first it was very odd and pretty funny but as the movie progressed I didn't find the jokes or oddness funny anymore. Its a low budget film (thats never a problem in itself), there were some pretty interesting characters, but eventually I just lost interest. I imagine this film would appeal to a stoner who is currently partaking. For something similar but better try ""Brother from another planet""",phil the alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines at first it was very odd and pretty funny but as the movie progressed i didnt find the jokes or oddness funny anymore its a low budget film thats never a problem in itself there were some pretty interesting characters but eventually i just lost interest i imagine this film would appeal to a stoner who is currently partaking for something similar but better try brother from another planet,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   review        7000 non-null   object
 1   review_clean  7000 non-null   object
 2   label         7000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 164.2+ KB


## Stopword Removal

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
stop_words = stopwords.words('english')
print(stop_words)
print(len(stop_words))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
def removeStopword(df,stop_words_):
    for stop_word in stop_words_:
        regex_stopword = r"\b" + stop_word + r"\b"
        df = df.str.replace(regex_stopword, '')
    return df

In [10]:
data['review_stopword'] = removeStopword(data['review_clean'],stop_words)
data = data[['label','review','review_clean','review_stopword']]
data.head(3)

,label,review,review_clean,review_stopword
0,0,"Encouraged by the positive comments about this film on here I was looking forward to watching this film. Bad mistake. I've seen 950+ films and this is truly one of the worst of them - it's awful in almost every way: editing, pacing, storyline, 'acting,' soundtrack (the film's only song - a lame country tune - is played no less than four times). The film looks cheap and nasty and is boring in the extreme. Rarely have I been so happy to see the end credits of a film. The only thing that prevents me giving this a 1-score is Harvey Keitel - while this is far from his best performance he at least seems to be making a bit of an effort. One for Keitel obsessives only.",encouraged by the positive comments about this film on here i was looking forward to watching this film bad mistake ive seen films and this is truly one of the worst of them its awful in almost every way editing pacing storyline acting soundtrack the films only song a lame country tune is played no less than four times the film looks cheap and nasty and is boring in the extreme rarely have i been so happy to see the end credits of a film the only thing that prevents me giving this a score is harvey keitel while this is far from his best performance he at least seems to be making a bit of an effort one for keitel obsessives only,encouraged positive comments film looking forward watching film bad mistake ive seen films truly one worst awful almost every way editing pacing storyline acting soundtrack films song lame country tune played less four times film looks cheap nasty boring extreme rarely happy see end credits film thing prevents giving score harvey keitel far best performance least seems making bit effort one keitel obsessives
1,1,"If you like original gut wrenching laughter you will like this movie. If you are young or old then you will love this movie, hell even my mom liked it. Great Camp!!!",if you like original gut wrenching laughter you will like this movie if you are young or old then you will love this movie hell even my mom liked it great camp,like original gut wrenching laughter like movie young old love movie hell even mom liked great camp
2,0,"Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines. At first it was very odd and pretty funny but as the movie progressed I didn't find the jokes or oddness funny anymore. Its a low budget film (thats never a problem in itself), there were some pretty interesting characters, but eventually I just lost interest. I imagine this film would appeal to a stoner who is currently partaking. For something similar but better try ""Brother from another planet""",phil the alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines at first it was very odd and pretty funny but as the movie progressed i didnt find the jokes or oddness funny anymore its a low budget film thats never a problem in itself there were some pretty interesting characters but eventually i just lost interest i imagine this film would appeal to a stoner who is currently partaking for something similar but better try brother from another planet,phil alien one quirky films humour based around oddness everything rather actual punchlines first odd pretty funny movie progressed didnt find jokes oddness funny anymore low budget film thats never problem pretty interesting characters eventually lost interest imagine film would appeal stoner currently partaking something similar better try brother another planet


## Lemmatization

In [11]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [12]:
stemmer_en = WordNetLemmatizer() # nltk

In [13]:
def stemming(text, stemmer_en):
    text_split = text.split(" ")
    stemmed_list = []
    for i in text_split:
        stem_text1 = stemmer_en.lemmatize(i, pos="v")
        stem_text2 = stemmer_en.lemmatize(stem_text1, pos="n")
        stem_text3 = stemmer_en.lemmatize(stem_text2, pos="a")
        stemmed_list.append(stem_text3)
    stemmed = ' '.join(map(str,stemmed_list))
    return stemmed

def stemData(data, stemmer_en):
    data = data.map(lambda com : stemming(com,stemmer_en))
    return data

In [14]:
data['review_stem'] = stemData(data['review_stopword'], stemmer_en)
data.head(3)

,label,review,review_clean,review_stopword,review_stem
0,0,"Encouraged by the positive comments about this film on here I was looking forward to watching this film. Bad mistake. I've seen 950+ films and this is truly one of the worst of them - it's awful in almost every way: editing, pacing, storyline, 'acting,' soundtrack (the film's only song - a lame country tune - is played no less than four times). The film looks cheap and nasty and is boring in the extreme. Rarely have I been so happy to see the end credits of a film. The only thing that prevents me giving this a 1-score is Harvey Keitel - while this is far from his best performance he at least seems to be making a bit of an effort. One for Keitel obsessives only.",encouraged by the positive comments about this film on here i was looking forward to watching this film bad mistake ive seen films and this is truly one of the worst of them its awful in almost every way editing pacing storyline acting soundtrack the films only song a lame country tune is played no less than four times the film looks cheap and nasty and is boring in the extreme rarely have i been so happy to see the end credits of a film the only thing that prevents me giving this a score is harvey keitel while this is far from his best performance he at least seems to be making a bit of an effort one for keitel obsessives only,encouraged positive comments film looking forward watching film bad mistake ive seen films truly one worst awful almost every way editing pacing storyline acting soundtrack films song lame country tune played less four times film looks cheap nasty boring extreme rarely happy see end credits film thing prevents giving score harvey keitel far best performance least seems making bit effort one keitel obsessives,encourage positive comment film look forward watch film bad mistake ive see film truly one bad awful almost every way edit pace storyline act soundtrack film song lame country tune play le four time film look cheap nasty bore extreme rarely happy see end credit film thing prevent give score harvey keitel far best performance least seem make bite effort one keitel obsessive
1,1,"If you like original gut wrenching laughter you will like this movie. If you are young or old then you will love this movie, hell even my mom liked it. Great Camp!!!",if you like original gut wrenching laughter you will like this movie if you are young or old then you will love this movie hell even my mom liked it great camp,like original gut wrenching laughter like movie young old love movie hell even mom liked great camp,like original gut wrench laughter like movie young old love movie hell even mom like great camp
2,0,"Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines. At first it was very odd and pretty funny but as the movie progressed I didn't find the jokes or oddness funny anymore. Its a low budget film (thats never a problem in itself), there were some pretty interesting characters, but eventually I just lost interest. I imagine this film would appeal to a stoner who is currently partaking. For something similar but better try ""Brother from another planet""",phil the alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines at first it was very odd and pretty funny but as the movie progressed i didnt find the jokes or oddness funny anymore its a low budget film thats never a problem in itself there were some pretty interesting characters but eventually i just lost interest i imagine this film would appeal to a stoner who is currently partaking for something similar but better try brother from another planet,phil alien one quirky films humour based around oddness everything rather actual punchlines first odd pretty funny movie progressed didnt find jokes oddness funny anymore low budget film thats never problem pretty interesting characters eventually lo

In [15]:
data_prep_stem = data[['review_stem','label']]
data_prep_stem = data_prep_stem.rename(columns={'review' : 'review_stem'})

data_prep_stem.to_csv("data_prep_stem.csv")